In [1]:
! pip install -e .

Obtaining file:///
ERROR: file:/// does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
input="/app/data/outputs/2024.09.22/07.35.36_train_diffusion_unet_hybrid_pusht_image/checkpoints/latest.ckpt"
output="/app/data/outputs/2024.09.22/07.35.36_train_diffusion_unet_hybrid_pusht_image/checkpoints/output"
robot_ip="192.168.0.204"
match_dataset="/app/data/pusht_real/real_pusht_20230105"
match_episode=None
vis_camera_idx=0
init_joints=False
steps_per_inference=6
max_duration=60
frequency=10
command_latency=0.01

In [3]:
import sys
sys.path.append("/opt/ros/noetic/lib/python3/dist-packages")
sys.path.append("/app/diffusion_policy/real_world/dynamic_biped_ws/devel/lib/python3/dist-packages")


In [4]:
!source /app/diffusion_policy/real_world/dynamic_biped_ws/devel/setup.zsh

In [5]:
import rospkg
import rospy

In [6]:
import time
from multiprocessing.managers import SharedMemoryManager
import click
import cv2
import numpy as np
import torch
import dill
import hydra
import pathlib
import skvideo.io
from omegaconf import OmegaConf
import scipy.spatial.transform as st
from diffusion_policy.real_world.real_env_kuavo import KuavoEnv
# from diffusion_policy.real_world.spacemouse_shared_memory import Spacemouse
from diffusion_policy.common.precise_sleep import precise_wait
from diffusion_policy.real_world.real_inference_util import (
    get_real_obs_resolution, 
    get_real_obs_dict)
from diffusion_policy.common.pytorch_util import dict_apply
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.base_image_policy import BaseImagePolicy
from diffusion_policy.common.cv2_util import get_image_transform

/root/miniconda3/envs/robodiff/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# load match_dataset
match_camera_idx = 0
episode_first_frame_map = dict()
if match_dataset is not None:
    match_dir = pathlib.Path(match_dataset)
    match_video_dir = match_dir.joinpath('videos')
    for vid_dir in match_video_dir.glob("*/"):
        episode_idx = int(vid_dir.stem)
        match_video_path = vid_dir.joinpath(f'{match_camera_idx}.mp4')
        if match_video_path.exists():
            frames = skvideo.io.vread(
                str(match_video_path), num_frames=1)
            episode_first_frame_map[episode_idx] = frames[0]
        break
print(f"Loaded initial frame for {len(episode_first_frame_map)} episodes")

Loaded initial frame for 1 episodes


In [8]:
import os
os.environ['http_proxy'] = 'http://172.17.0.1:7890'
os.environ['https_proxy'] = 'http://172.17.0.1:7890'



In [40]:
# load checkpoint
ckpt_path = input
payload = torch.load(open(ckpt_path, 'rb'), pickle_module=dill)
cfg = payload['cfg']
cls = hydra.utils.get_class(cfg._target_)
workspace = cls(cfg)
workspace: BaseWorkspace
workspace.load_payload(payload, exclude_keys=None, include_keys=None)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['agent_pos']
using obs modality: rgb with keys: ['image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
Diffusion params: 6.502567e+07
Vision params: 1.119709e+07


In [41]:
# hacks for method-specific setup.
action_offset = 0
delta_action = False
if 'diffusion' in cfg.name:
    # diffusion model
    policy: BaseImagePolicy
    policy = workspace.model
    if cfg.training.use_ema:
        policy = workspace.ema_model
    device = torch.device('cuda')
    policy.eval().to(device)
    # set inference params
    policy.num_inference_steps = 16 # DDIM inference iterations
    policy.n_action_steps = 32
elif 'robomimic' in cfg.name:
    # BCRNN model
    policy: BaseImagePolicy
    policy = workspace.model
    device = torch.device('cuda')
    policy.eval().to(device)
    # BCRNN always has action horizon of 1
    steps_per_inference = 1
    action_offset = cfg.n_latency_steps
    delta_action = cfg.task.dataset.get('delta_action', False)
elif 'ibc' in cfg.name:
    policy: BaseImagePolicy
    policy = workspace.model
    policy.pred_n_iter = 5
    policy.pred_n_samples = 4096
    device = torch.device('cuda')
    policy.eval().to(device)
    steps_per_inference = 1
    action_offset = 1
    delta_action = cfg.task.dataset.get('delta_action', False)
else:
    raise RuntimeError("Unsupported policy type: ", cfg.name)

In [42]:
# setup experiment
dt = 1/frequency
print(cfg)
obs_res = get_real_obs_resolution(cfg.task.shape_meta)
n_obs_steps = cfg.n_obs_steps
print("n_obs_steps: ", n_obs_steps)
print("steps_per_inference:", steps_per_inference)
print("action_offset:", action_offset)
print('n_action_steps', cfg.n_action_steps)

{'_target_': 'diffusion_policy.workspace.train_diffusion_unet_hybrid_workspace.TrainDiffusionUnetHybridWorkspace', 'checkpoint': {'save_last_ckpt': True, 'save_last_snapshot': False, 'topk': {'format_str': 'epoch={epoch:04d}-train_loss={train_loss:.3f}.ckpt', 'k': 5, 'mode': 'min', 'monitor_key': 'train_loss'}}, 'dataloader': {'batch_size': 128, 'num_workers': 8, 'persistent_workers': False, 'pin_memory': True, 'shuffle': True}, 'dataset_obs_steps': 2, 'ema': {'_target_': 'diffusion_policy.model.diffusion.ema_model.EMAModel', 'inv_gamma': 1.0, 'max_value': 0.9999, 'min_value': 0.0, 'power': 0.75, 'update_after_step': 0}, 'exp_name': 'default', 'horizon': 16, 'keypoint_visible_rate': 1.0, 'logging': {'group': None, 'id': None, 'mode': 'online', 'name': '2023.01.16-20.20.06_train_diffusion_unet_hybrid_pusht_image', 'project': 'diffusion_policy_debug', 'resume': True, 'tags': ['train_diffusion_unet_hybrid', 'pusht_image', 'default']}, 'multi_run': {'run_dir': 'data/outputs/2023.01.16/20.2

In [43]:
env = KuavoEnv(
        output_dir=output,
        frequency=frequency,
        n_obs_steps=2,
        obs_image_resolution=(640, 480),
        max_obs_buffer_size=30,
        robot_publish_rate=125,
        video_capture_fps=30,
        video_capture_resolution=(1280, 720),
        )
    

In [44]:
fake_obs=env.get_fake_obs()
for k, v in fake_obs.items():
    print(k, v.shape)

image (2, 480, 640, 3)
agent_pos (2, 7)
timestamp (2,)


In [45]:
try:
    # start episode
    policy.reset()
    start_delay = 1.0
    eval_t_start = time.time() + start_delay
    t_start = time.monotonic() + start_delay
    # env.start_episode(eval_t_start)
    # wait for 1/30 sec to get the closest frame actually
    # reduces overall latency
    frame_latency = 1/30
    precise_wait(eval_t_start - frame_latency, time_func=time.time)
    print("Started!")
    iter_idx = 0
    term_area_start_timestamp = float('inf')
    perv_target_pose = None
    while True:
        # calculate timing
        t_cycle_end = t_start + (iter_idx + steps_per_inference) * dt
        # get obs
        print('get_obs')
        # obs = env.get_obs()
        obs = env.get_fake_obs()
        obs_timestamps = obs['timestamp']
        print(f'Obs latency {time.time() - obs_timestamps[-1]}')
        # run inference
        with torch.no_grad():
            s = time.time()
            obs_dict_np = get_real_obs_dict(
                env_obs=obs, shape_meta=cfg.task.shape_meta)
            obs_dict = dict_apply(obs_dict_np, 
                lambda x: torch.from_numpy(x).unsqueeze(0).to(device))
            result = policy.predict_action(obs_dict)
            # this action starts from the first obs step
            action = result['action'][0].detach().to('cpu').numpy()
            print('Inference latency:', time.time() - s)
        
        # # clip actions
        # this_target_poses[:,:2] = np.clip(
        #     this_target_poses[:,:2], [0.25, -0.45], [0.77, 0.40])
        # execute actions
        print(action.shape)
        env.exec_fake_actions(
            actions=action,
        )
        break
        
        # print(f"Submitted {len(action)} steps of actions.")
        # # visualize
        # episode_id = env.replay_buffer.n_episodes
        # vis_img = obs[f'camera_{vis_camera_idx}'][-1]
        # text = 'Episode: {}, Time: {:.1f}'.format(
        #     episode_id, time.monotonic() - t_start
        # )
        # cv2.putText(
        #     vis_img,
        #     text,
        #     (10,20),
        #     fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        #     fontScale=0.5,
        #     thickness=1,
        #     color=(255,255,255)
        # )
        # cv2.imshow('default', vis_img[...,::-1])
        # key_stroke = cv2.pollKey()
        # if key_stroke == ord('s'):
        #     # Stop episode
        #     # Hand control back to human
        #     env.end_episode()
        #     print('Stopped.')
        #     break
        # # auto termination
        # terminate = False
        # if time.monotonic() - t_start > max_duration:
        #     terminate = True
        #     print('Terminated by the timeout!')
        # term_pose = np.array([ 3.40948500e-01,  2.17721816e-01,  4.59076878e-02,  2.22014183e+00, -2.22184883e+00, -4.07186655e-04])
        # curr_pose = obs['robot_eef_pose'][-1]
        # dist = np.linalg.norm((curr_pose - term_pose)[:2], axis=-1)
        # if dist < 0.03:
        #     # in termination area
        #     curr_timestamp = obs['timestamp'][-1]
        #     if term_area_start_timestamp > curr_timestamp:
        #         term_area_start_timestamp = curr_timestamp
        #     else:
        #         term_area_time = curr_timestamp - term_area_start_timestamp
        #         if term_area_time > 0.5:
        #             terminate = True
        #             print('Terminated by the policy!')
        # else:
        #     # out of the area
        #     term_area_start_timestamp = float('inf')
        # if terminate:
        #     env.end_episode()
        #     break
        # # wait for execution
        # precise_wait(t_cycle_end - frame_latency)
        # iter_idx += steps_per_inference
except KeyboardInterrupt:
    print("Interrupted!")
    # stop robot.
    env.end_episode()

Started!
get_obs
Obs latency -0.08899044990539551
1 32
Inference latency: 0.10473346710205078
(15, 7)
executing actions: (15, 7)
